#FlipItNews: Natural Language Processing Business Case


Goal:


---


The goal here is to help FlipItNews to
intelligently categorize content into domains like politics, technology, sports,
business, and entertainment by analyzing its internal database of news article.

Benefit:


---


The insights from this analysis can significantly improve content discovery and
user engagement, driving financial literacy and investment awareness among
millennials and first-time investors.



In [1]:
!git clone https://github.com/yedhuk/categorizing-news-articles-flipitnews.git

Cloning into 'categorizing-news-articles-flipitnews'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), done.


In [2]:
%cd categorizing-news-articles-flipitnews

/content/categorizing-news-articles-flipitnews


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mv "" "/content/categorizing-news-articles-flipitnews/notebook"

mv: cannot stat '/content/FlipItNews Business Case.ipynb': No such file or directory


In [ ]:
!gdown 17cj01qd7xyY8RmUhH0pMoYf6z53dEQWW

In [ ]:
import numpy as np
import pandas as pd